# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install openai==0.28

In [3]:
!ls drive/MyDrive/Colab\ Notebooks

 7k-notesbook
 7k.zip
 airbnb-nyc-price-prediction.ipynb
 airbnb-price-prediction-using-xgboost.ipynb
 compressed.png
'Copy of Copy of 2_Simple_encoder_decoder_Medical_Report.ipynb'
'Copy of Copy of Pneumonia Classification with notes .ipynb'
'Copy of HW2_Q2_running.ipynb'
'Copy of try_cot_colab.ipynb'
 Dataset4K
'data structures'
 dsc250-project-llm-for-adhd-CoT-prompting.ipynb
 dsc253-hw1
 dsc253.ipynb
 Kodak
 openai-secret.txt
 original.png
 sample.csv
 simple_baseline-feb28.ipynb
 SVD.ipynb
 TDA-project
 Untitled0.ipynb
 xgboost_kaggle_2.ipynb
'xgboost_kaggle_2-original copy.ipynb'


In [48]:
!cd
!ls

api.py		    demos    __pycache__       run_demo.py	    try_cot.ipynb
CODE_OF_CONDUCT.md  LICENSE  README.md	       run_inference.py     utils.py
CONTRIBUTING.md     NOTICE   requirements.txt  try_cot_colab.ipynb


In [16]:
import os
import openai

# openai_key = open('drive/MyDrive/Colab Notebooks/openai-secret.txt').read()
# os.environ["OPENAI_API_KEY"] = openai_key

openai.api_key_path = 'drive/MyDrive/Colab Notebooks/openai-secret.txt'

response = openai.Completion.create(
    engine="text-davinci-002",  # Choose the appropriate engine/model
    prompt="Translate the following English text to French: '{}'",
    max_tokens=60
)



FileNotFoundError: ignored

In [5]:
response

<OpenAIObject text_completion id=cmpl-8MTJNBgB53EGctcw55IzWfCMaNLfW at 0x7b5e401f5300> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8MTJNBgB53EGctcw55IzWfCMaNLfW",
  "object": "text_completion",
  "created": 1700366581,
  "model": "text-davinci-002",
  "choices": [
    {
      "text": "\n\nJ'ai d\u00e9cid\u00e9 de m'inscrire \u00e0 des le\u00e7ons de fran\u00e7ais pour am\u00e9liorer mon niveau de langue.\n\nI have decided to sign up for French lessons to improve my language level.",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 13,
    "completion_tokens": 56,
    "total_tokens": 69
  }
}

# Sample CoT Usage

In [6]:
!git clone https://github.com/amazon-science/auto-cot.git
%cd auto-cot

fatal: destination path 'auto-cot' already exists and is not an empty directory.
/content/auto-cot


In [7]:
!pip install -r requirements.txt
!pip install openai

## API function

In [90]:
import argparse
from utils import *

def cot(method, question, printOut = True):
    args = parse_arguments()
    decoder = Decoder()

    args.method = method
    if args.method != "zero_shot_cot":
        if args.method == "auto_cot":
            args.demo_path = "demos/multiarith_auto"
        else:
            args.demo_path = "demos/multiarith_manual"
        demo = create_demo_text(args, cot_flag=True)
    else:
        demo = None

    x = "Q: " + question + "\n" + "A:"
    if printOut:
      print('*****************************')
      print("Test Question:")
      print(question)
      print('*****************************')

    if args.method == "zero_shot":
        x = x + " " + args.direct_answer_trigger_for_zeroshot
    elif args.method == "zero_shot_cot":
        x = x + " " + args.cot_trigger
    elif args.method == "manual_cot":
        x = demo + x
    elif args.method == "auto_cot":
        x = demo + x + " " + args.cot_trigger
    else:
        raise ValueError("method is not properly defined ...")

    prompted_input = x.replace("\n\n", "\n").strip()
    if printOut:
      print("Prompted Input:")
      print(x.replace("\n\n", "\n").strip())
      print('*****************************')

    max_length = args.max_length_cot if "cot" in args.method else args.max_length_direct
    z = decoder.decode(args, x, max_length)
    z = z.replace("\n\n", "\n").replace("\n", "").strip()
    if args.method == "zero_shot_cot":
        z2 = x + z + " " + args.direct_answer_trigger_for_zeroshot_cot
        max_length = args.max_length_direct
        pred = decoder.decode(args, z2, max_length)

        output = z + " " + args.direct_answer_trigger_for_zeroshot_cot + " " + pred
        if printOut:
          print("Output:")
          print(z + " " + args.direct_answer_trigger_for_zeroshot_cot + " " + pred)
          print('*****************************')
    else:
        pred = z
        output = pred
        if printOut:
          print("Output:")
          print(pred)
          print('*****************************')

    return ({'method': method,
            'prompted_input': prompted_input,
            'output': output})


def parse_arguments():
    parser = argparse.ArgumentParser(description="Zero-shot-CoT")

    parser.add_argument("--max_num_worker", type=int, default=0, help="maximum number of workers for dataloader")
    parser.add_argument(
        "--model", type=str, default="gpt3-xl", help="model used for decoding. Note that 'gpt3' are the smallest models."
    )
    parser.add_argument(
        "--method", type=str, default="auto_cot", choices=["zero_shot", "zero_shot_cot", "few_shot", "few_shot_cot", "auto_cot"], help="method"
    )
    parser.add_argument(
        "--cot_trigger_no", type=int, default=1, help="A trigger sentence that elicits a model to execute chain of thought"
    )
    parser.add_argument(
        "--max_length_cot", type=int, default=256, help="maximum length of output tokens by model for reasoning extraction"
    )
    parser.add_argument(
        "--max_length_direct", type=int, default=32, help="maximum length of output tokens by model for answer extraction"
    )
    parser.add_argument(
        "--limit_dataset_size", type=int, default=0, help="whether to limit test dataset size. if 0, the dataset size is unlimited and we use all the samples in the dataset for testing."
    )
    parser.add_argument(
        "--api_time_interval", type=float, default=1.0, help=""
    )
    parser.add_argument(
        "--temperature", type=float, default=0, help=""
    )
    parser.add_argument(
        "--log_dir", type=str, default="./log/", help="log directory"
    )
    args = parser.parse_args()

    args.direct_answer_trigger_for_fewshot = "The answer is"
    args.direct_answer_trigger_for_zeroshot = "The answer is"
    args.direct_answer_trigger_for_zeroshot_cot = "The answer is"
    args.cot_trigger = "Let's think step by step."

    return args

## Zero Shot

In [74]:
openai.api_key_path = '/content/openai-secret.txt'

import sys
sys.argv=['']
del sys
# from api import cot

question = "There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?"
print("Example: Zero-Shot")
# To use GPT-3, please add your openai-api key in utils.py (#Line 59)
# method = ["zero_shot", "zero_shot_cot", "manual_cot", "auto_cot"]
cot(method="zero_shot", question=question)



Example: Zero-Shot
*****************************
Test Question:
There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
*****************************
Prompted Input:
Q: There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
A: The answer is
*****************************
Output:
80.
*****************************


{'method': 'zero_shot',
 'prompted_input': 'Q: There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?\nA: The answer is',
 'output': '80.'}

In [21]:
question = "There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?"
print("Example: Zero-Shot-CoT")
cot(method="zero_shot_cot", question=question)

Example: Zero-Shot-CoT
*****************************
Test Question:
There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
*****************************
Prompted Input:
Q: There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
A: Let's think step by step.
*****************************
Output:
There were 10 friends playing a video game online. This means that, at the start, there were 10 x 8 = 80 lives in total. Then, 7 players quit. This means that 7 x 8 = 56 lives were lost. Therefore, the total number of lives remaining is 80 - 56 = 24. The answer is  24.
*****************************


## Manual CoT

In [22]:
question = "In a video game, each enemy defeated gives you 7 points. If a level has 11 enemies total and you destroy all but 8 of them, how many points would you earn?"
print("Example: Manual-CoT")
cot(method="manual_cot", question=question)

Example: Manual-CoT
*****************************
Test Question:
In a video game, each enemy defeated gives you 7 points. If a level has 11 enemies total and you destroy all but 8 of them, how many points would you earn?
*****************************
Prompted Input:
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they 

## Auto CoT

In [23]:
question = "In a video game, each enemy defeated gives you 7 points. If a level has 11 enemies total and you destroy all but 8 of them, how many points would you earn?"
print("Example: Auto-CoT")
cot(method="auto_cot", question=question)

Example: Auto-CoT
*****************************
Test Question:
In a video game, each enemy defeated gives you 7 points. If a level has 11 enemies total and you destroy all but 8 of them, how many points would you earn?
*****************************
Prompted Input:
Q: Wendy uploaded 45 pictures to Facebook. She put 27 pics into one album and put the rest into 9 different albums. How many pictures were in each album?
A: Let's think step by step. First, we know that Wendy uploaded 45 pictures in total. Second, we know that Wendy put 27 pictures into one album. That means that Wendy put the remaining 18 pictures into 9 different albums. That means that each album would have 2 pictures. The answer is 2.
Q: For Halloween Katie and her sister combined the candy they received. Katie had 8 pieces of candy while her sister had 23. If they ate 8 pieces the first night, how many pieces do they have left?
A: Let's think step by step. Katie and her sister have a total of 8 + 23 = 31 pieces of candy.

# ADHD

In [48]:
import pandas as pd

# URL to the CSV file
csv_url = "/content/drive/MyDrive/Colab Notebooks/sample.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_url, encoding='unicode_escape')

# Display the first few rows of the DataFrame
print(df.head())
print(len(df))

                                            question
0  Does the individual often have difficulty sust...
1  Is there a tendency to make careless mistakes ...
2  Does the person seem to be easily distracted b...
3  Is there a frequent forgetfulness in daily act...
4  Does the individual have difficulty organizing...
40


In [38]:
from random import randrange

randomIndex = randrange(19)
print("Question number", randomIndex + 1)
question = df[0][randomIndex]

print("Example: Zero-Shot")
cot(method="zero_shot", question=question)

print("Example: Zero-Shot-CoT")
cot(method="zero_shot_cot", question=question)


print("Example: Manual-CoT")
cot(method="manual_cot", question=question)

print("Example: Auto-CoT")
cot(method="auto_cot", question=question)


Question number 9
Example: Zero-Shot
*****************************
Test Question:
Does the individual often act without thinking, resulting in accidents or impulsive decisions?
*****************************
Prompted Input:
Q: Does the individual often act without thinking, resulting in accidents or impulsive decisions?
A: The answer is
*****************************
Output:
yes.
*****************************
Example: Zero-Shot-CoT
*****************************
Test Question:
Does the individual often act without thinking, resulting in accidents or impulsive decisions?
*****************************
Prompted Input:
Q: Does the individual often act without thinking, resulting in accidents or impulsive decisions?
A: Let's think step by step.
*****************************
Output:
First, does the individual in question often act without thinking? This could mean that they make decisions impulsively, without considering the consequences, or that they act without considering the potential for a

In [ ]:
methods = ["zero_shot", "zero_shot_cot", "manual_cot", "auto_cot"]


for method in methods:
    # Apply the cot function to each row and convert the result to a DataFrame
    method_results = df['question'].apply(lambda x: pd.Series(cot(method, x, printOut=False)))

    # Add the new columns to the original DataFrame
    df = pd.concat([df, method_results.add_prefix(f'{method}_')], axis=1)

In [108]:
df

,question,zero_shot_prompted_input,zero_shot_output,zero_shot_cot_prompted_input,zero_shot_cot_output,manual_cot_prompted_input,manual_cot_output,auto_cot_prompted_input,auto_cot_output
0,Does the individual often have difficulty sust...,Q: Does the individual often have difficulty s...,yes.,Q: Does the individual often have difficulty s...,"First, does the individual often have difficul...",Q: There are 15 trees in the grove. Grove work...,This question is difficult to answer without m...,Q: Wendy uploaded 45 pictures to Facebook. She...,The individual often has difficulty sustaining...
1,Is there a tendency to make careless mistakes ...,Q: Is there a tendency to make careless mistak...,yes.,Q: Is there a tendency to make careless mistak...,"First, inattention can make it difficult to fo...",Q: There are 15 trees in the grove. Grove work...,There is a tendency to make careless mistakes ...,Q: Wendy uploaded 45 pictures to Facebook. She...,We know that there are a lot of distractions i...
2,Does the person seem to be easily distracted b...,Q: Does the person seem to be easily distracte...,difficult to determine from the information gi...,Q: Does the person seem to be easily distracte...,"First, we need to determine what ""unrelated st...",Q: There are 15 trees in the grove. Grove work...,"No, the person does not seem to be easily dist...",Q: Wendy uploaded 45 pictures to Facebook. She...,The person seems to be easily distracted by un...
3,Is there a frequent forgetfulness in daily act...,Q: Is there a frequent forgetfulness in daily ...,unclear.,Q: Is there a frequent forgetfulness in daily ...,"First, we need to consider what ""frequent forg...",Q: There are 15 trees in the grove. Grove work...,There is no right or wrong answer to this ques...,Q: Wendy uploaded 45 pictures to Facebook. She...,The question is asking if there is a frequent ...
4,Does the individual have difficulty organizing...,Q: Does the individual have difficulty organiz...,yes.,Q: Does the individual have difficulty organiz...,The first thing you need to do is break the ta...,Q: There are 15 trees in the grove. Grove work...,This question is difficult to answer without m...,Q: Wendy uploaded 45 pictures to Facebook. She...,If the individual has difficulty organizing ta...
5,Is there avoidance or reluctance to engage in ...,Q: Is there avoidance or reluctance to engage ...,yes.,Q: Is there avoidance or reluctance to engage ...,"First, we need to come up with a definition fo...",Q: There are 15 trees in the grove. Grove work...,There may be avoidance or reluctance to engage...,Q: Wendy uploaded 45 pictures to Facebook. She...,The answer is yes.
6,Does the person often fidget or tap hands or f...,Q: Does the person often fidget or tap hands o...,yes.,Q: Does the person often fidget or tap hands o...,The person is seated.The person fidget or tap ...,Q: There are 15 trees in the grove. Grove work...,This is not a math question.,Q: Wendy uploaded 45 pictures to Facebook. She...,The person often fidget or tap hands or feet w...
7,"Is there a tendency to talk excessively, inter...","Q: Is there a tendency to talk excessively, in...",yes.,"Q: Is there a tendency to talk excessively, in...","First, we need to consider what it means to ta...",Q: There are 15 trees in the grove. Grove work...,"There is a tendency to talk excessively, inter...",Q: Wendy uploaded 45 pictures to Facebook. She...,This question is asking if there is a tendency...
8,Does the individual often act without thinking...,Q: Does the individual often act without think...,yes.,Q: Does the individual often act without think...,"First, does the individual in question often a...",Q: There are 15 trees in the grove. Grove work...,This question is difficult to answer without m...,Q: Wendy uploaded 45 pictures to Facebook. She...,The individual often acts without thinking. Th...
9,Are there difficulties in staying seated in si...,Q: Are there difficulties in staying seated in...,"yes, there can be difficu

In [106]:
df.to_csv("result.csv", index=False)